In [79]:
import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
import numpy as np
import datetime
df=pd.read_excel(r'DataSource/flights.xlsx')
for col in ['更新时间','来源','频率','单位']:
    if col in df.columns:
        df=df.drop(columns=col)
df=df.dropna(subset=['指标名称'])
c1=df['指标名称'].str.contains('数据来源')
df=df[~c1]
df['指标名称']=df['指标名称'].str.replace('机场进出港航班:','')
df[['城市', '机场']] = df['指标名称'].str.split("/", expand = True)
df=df.drop(columns=['指标名称'])
cols=list(df.columns[-2:])+sorted(list(df.columns[:-2]),reverse=False)
df=df[cols]
df.to_hdf('./Processed_Data/airports.h5',key='raw')

ap=pd.read_hdf('./Processed_Data/airports.h5',key='raw')
ap=ap.groupby(by='城市').sum().reset_index()
lastcol=ap.columns[-1]
ap=ap.sort_values(by=lastcol,ascending=False).head(50)
ap=pd.melt(ap,id_vars=['城市'],var_name='date',value_name='air_traffic')
ap=ap[ap['air_traffic']>0]

ap['date'] = pd.to_datetime(ap['date']).dt.date
ap=ap.sort_values(by=['城市','date'],ascending=True)
ap['date']=ap['date'].astype('str')
ap['dt']=ap['date'].str[-5:]

ap['ap_last7d_mean']=ap.groupby('城市')['air_traffic'].transform(lambda x: x.rolling(7, 1).mean())
ap['ap_WoW']=ap['ap_last7d_mean']/ap.groupby('城市')['ap_last7d_mean'].transform(lambda x: x.shift(periods=7)) -1

c1=ap['date']>='2018-11-01'
c2=ap['date']<='2019-10-01'
ap_19=ap[c1&c2]
ap_19=ap_19[['城市', 'dt','ap_last7d_mean']]
ap_19['ap_last7d_mean']=ap.groupby('城市')['ap_last7d_mean'].transform(lambda x: x.shift(periods=-14)) #调整CNY
ap_19=ap_19.rename(columns={'ap_last7d_mean':'ap_19_rolling_mean'})

c2=ap['date']>'2022-05-01'
ap_22=ap[c2]
ap_22=ap_22.merge(ap_19,how='left',on=['城市','dt'])

ap_22['ap_status']=ap_22['ap_last7d_mean']/ap_22['ap_19_rolling_mean']

In [80]:
# TY 机场数据 WoW vs. GDP
riqi='2023-01-15'
city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
o=ap_22.merge(city_popu,on=['城市'],how='left')

o=o[o['date']==riqi][['城市','GDP','ap_WoW']] 

o['size']=np.nan
o['group']='其他'

o=o.sort_values(by=['group','GDP'],ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-15


In [81]:
# TY 机场数据 as of 19 vs. GDP

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
o=ap_22.merge(city_popu,on=['城市'],how='left')

o=o[o['date']==riqi][o['ap_status']<1.5][['城市','GDP','ap_status']] 

o['size']=np.nan
o['group']='其他'

o=o.sort_values(by=['group','GDP'],ascending=False)

d=ap_22[ap_22['date']==riqi][['城市','date','ap_status']].reset_index(drop=True).rename(columns={'ap_status':'airports'})
d['date']=pd.to_datetime(d['date'])
d.to_hdf('Processed_Data/city_compare.h5',key='airports')


o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-15


In [75]:
# d1=pd.read_clipboard()
# d1.to_hdf('Processed_Data/tablehead.h5','airports')
d1=pd.read_hdf('Processed_Data/tablehead.h5','airports')

apo=pd.read_hdf('./Processed_Data/airports.h5',key='raw')
cols=[i for i in apo.columns if i >='2023-01-11']

d1.merge(apo[cols],on=['城市','机场'],how='left').drop(columns=['城市','机场']).to_clipboard(index=False)

In [86]:

# citytier=pd.read_hdf('./Processed_Data/citypopu.h5',key='citytier')


# bytier=ap_22.merge(citytier,on='城市',how='left')
# bytier['ap_last7d_mean']=bytier['ap_last7d_mean']*7
# bytier['ap_19_rolling_mean']=bytier['ap_19_rolling_mean']*7
# bytier['CityTier']=bytier['CityTier'].str.replace('四线城市','三线城市')
# bytier=bytier.groupby(['CityTier','date']).sum().reset_index()[['CityTier','date','ap_last7d_mean','ap_19_rolling_mean']]
# bytier['status']=bytier['ap_last7d_mean']/bytier['ap_19_rolling_mean']
# bytier['status']=bytier['status'].transform(lambda x: x.rolling(7, 1).mean())
# bytier=pd.pivot_table(bytier,index='date',columns='CityTier',values='status',aggfunc='mean').reset_index()
# bytier.sort_values(by='date').tail(20)

alldata=ap_22.copy()
alldata['ap_last7d_mean']=alldata['ap_last7d_mean']*7
alldata['ap_19_rolling_mean']=alldata['ap_19_rolling_mean']*7
alldata=alldata.groupby(['date']).sum().reset_index()[['date','ap_last7d_mean','ap_19_rolling_mean']]
alldata['status']=alldata['ap_last7d_mean']/alldata['ap_19_rolling_mean'] #CNY处理前面已经做过了
alldata['status']=alldata['status'].transform(lambda x: x.rolling(7, 1).mean())
alldata=alldata[['date','status']].rename(columns={'status':'all'})
alldata.tail(15).to_clipboard(index=False)

,date,all
244,2023-01-01,0.489078
245,2023-01-02,0.497540
246,2023-01-03,0.506560
247,2023-01-04,0.515544
248,2023-01-05,0.524251
249,2023-01-06,0.532227
250,2023-01-07,0.539785
251,2023-01-08,0.556293
252,2023-01-09,0.573056
253,2023-01-10,0.589940
